In [1]:
%display latex

En esta notebook implementamos algunos programas que nos permitan calcular la curvatura de un espacio riemanniano homogéneo reductivo $M = G/H$. Vamos a usar las fórmulas de [Bes08]. **Hay que tener cuidado de no usar mal las identificaciones.** 

Supondremos que $\dim M = n$, $\dim H = m$, y $G \subset SO(N)$ es un grupo de Lie conexo, semisimple y compacto de dimensión $m + n$. Consideraremos en $G$ la métrica biinvariante definida por
$$\langle X, Y \rangle^{bi} = - \operatorname{trace} (XY), \qquad X, Y \in \mathfrak g$$

La geometría de $M$ queda determinada por la descomposición reductiva
$$\mathfrak g = \mathfrak h \oplus \mathfrak m$$
en donde $\mathfrak m$ es el complemento ortogonal de $\mathfrak h$ con respecto a la métrica biinvariante en $\mathfrak g$ junto con un producto interno $\operatorname{Ad}(H)$ invariante en $\mathfrak m$ el cual denotamos por
$\langle X, Y\rangle$ para $X, Y \in \mathfrak m$. Usamos el mismo símbolo $\langle -, -\rangle$ para denotar la métrica $G$ invariante en $\mathfrak m$. También estamos suponiendo que la acción de $G$ en $M$ es (casi) efectiva. 

**OBS.** Cada $X \in \mathfrak g$ induce un campo de Killing $X^*$ en $M$ definiendo
$$X^*_p = \frac{d}{dt}\Big|_0 \operatorname{Exp}(tX) \cdot p$$

Esta asignación cambia el signo del corchete de Lie: para todos $X, Y \in \mathfrak g$ se tiene
$$[X, Y]^* = -[X^*, Y^*]$$

Y si denotamos por $o = eH$ tenemos que la asignación $\mathfrak m \to T_oM$ dada por $X \in \mathfrak m \mapsto X^*_o$ es una isometría lineal (más aún es la diferencial de la proyección al cociente en la identidad).

**Implementación.**
* Para introducir $\mathfrak g$ daremos una lista con una base $\mathfrak g$
* Para introducir $\mathfrak h$ daremos una lista con una base de $\mathfrak h$
* Implementaremos programas que verifiquen las listas mencionadas en los ítems anteriores efectivamente den lugar a un álgebra de Lie y una subálgebra de Lie
* Para determinar $\mathfrak m$ construiremos una base ortonormal de $\mathfrak g$ cuyos primeros $m$ elementos formen una base ortonormal de $\mathfrak h$. Luego los últimos $n$ elementos serán una base de $\mathfrak m$
* El producto interno en $\mathfrak m$ deberá ser introducido explícitamente y habra que implementar un programa que chequee la Ad-invariancia.

In [2]:
def lie_bracket(X, Y):
    return X * Y - Y * X

def biinv_metric(X, Y):
    return - (X * Y).trace()

def is_li(gens):
    n = len(gens)
    M = matrix([gens[i].list() for i in range(n)])
    if M.rank() == n:
        return True
    else:
        return False

def is_lie_algebra(basis):
    if is_li(basis) == False:
        print("Not a linear independent set")
    else:
        n = len(basis)
        M = matrix([basis[i].list() for i in range(n)])
        for i in range(n):
            for j in range(i+1, n):
                if matrix(M.rows() + [vector(lie_bracket(basis[i], basis[j]).list())]).rank() > n:
                    return False
        return True

## La esfera $S^2 = \operatorname{SO}(3) / \operatorname{SO}(2)$

In [3]:
%display latex

In [4]:
#Test round sphere
g_initial_basis = [
    matrix(SR,[[0,-1,0],[1,0,0],[0,0,0]]),
    matrix(SR,[[0,0,-1],[0,0,0],[1,0,0]]),
    matrix(SR,[[0,0,0],[0,0,-1],[0,1,0]]),
]
g_initial_basis

[
[ 0 -1  0]  [ 0  0 -1]  [ 0  0  0]
[ 1  0  0]  [ 0  0  0]  [ 0  0 -1]
[ 0  0  0], [ 1  0  0], [ 0  1  0]
]

In [5]:
is_lie_algebra(g_initial_basis)

True

In [6]:
h_initial_basis = g_initial_basis[:1]
h_initial_basis

[
[ 0 -1  0]
[ 1  0  0]
[ 0  0  0]
]

In [7]:
def is_subalgebra(h_basis, g_basis):
    if is_lie_algebra(h_basis):
        for X in h_basis:
            if is_li(g_basis + [X]):
                return False
        return True

In [8]:
is_subalgebra(h_initial_basis, g_initial_basis)

True

In [9]:
def gram_schdmidt(basis, metric):
    on_basis = []
    for i, X in enumerate(basis):
        Y = X
        for j in range(i):
            Y -= metric(X, on_basis[j]) * on_basis[j]
        on_basis.append(1/sqrt(metric(Y, Y)) * Y)
    return on_basis

In [10]:
g_adapted_basis = h_initial_basis.copy()
for X in g_initial_basis:
    if is_li(g_adapted_basis + [X]):
        g_adapted_basis.append(X)

g_on_basis = gram_schdmidt(g_adapted_basis, biinv_metric)
h_on_basis = g_on_basis[:len(h_initial_basis)]

In [11]:
g_on_basis == g_initial_basis

False

In [12]:
g_on_basis

[
[           0 -1/2*sqrt(2)            0]
[ 1/2*sqrt(2)            0            0]
[           0            0            0],

[           0            0 -1/2*sqrt(2)]
[           0            0            0]
[ 1/2*sqrt(2)            0            0],

[           0            0            0]
[           0            0 -1/2*sqrt(2)]
[           0  1/2*sqrt(2)            0]
]

In [13]:
m_initial_basis = g_on_basis[len(h_initial_basis):]
n = len(m_initial_basis)

In [14]:
def proj_h(X):
    return X - sum([biinv_metric(X, Y) * Y for Y in m_initial_basis])

def proj_m(X):
    return X - sum([biinv_metric(X, Y) * Y for Y in h_on_basis])

In [15]:
# Test con metrica normal homogenea
metric_m = biinv_metric

In [16]:
m_on_basis = gram_schdmidt(m_initial_basis, metric_m)
print(matrix(n, n, lambda i, j: metric_m(m_initial_basis[i], m_initial_basis[j])))
print("")
print(matrix(n, n, lambda i, j: metric_m(m_on_basis[i], m_on_basis[j])))

[1 0]
[0 1]

[1 0]
[0 1]


**Chequeamos la Ad-invariancia.** Por simplicidad trabajaremos al nivel del  álgebra de Lie verificando que $\operatorname{ad}_X |_{\mathfrak m}: \mathfrak m \to \mathfrak m$ sea antisimétrica para cada $X \in \mathfrak h$. Si denotamos por $X_1, \ldots, X_n$ la base ortonormal de $\mathfrak m$, entonces la matriz de $\operatorname{ad}_X$ en esta base esta dada por
$$(a_{ij}) = ( \langle [X, X_j], X_i\rangle)$$

In [17]:
def ad_matrix_m(X):
    return matrix(n, n, lambda i, j: metric_m(lie_bracket(X, m_on_basis[j]), m_on_basis[i]))

In [18]:
for X in h_on_basis:
    print(ad_matrix_m(X).is_skew_symmetric())

True


**Conexión de Levi-Civita:** para todos $X, Y \in \mathfrak m$, $$(\nabla_{X^*} Y^*)_o = - \frac12 [X, Y]_{\mathfrak m} + U(X, Y)$$ en donde $U: \mathfrak m \times \mathfrak m \to \mathfrak m$ está determinado implícitamente por la fórmula $$2 \langle U(X, Y), Z\rangle = \langle [Z, X]_{\mathfrak m}, Y \rangle + \langle X, [Z, Y]_{\mathfrak m}\rangle$$ para todo $Z \in \mathfrak m$


In [19]:
def U_tensor(X, Y):
    return (1/2) * sum([(metric_m(proj_m(lie_bracket(Z,X)),Y) + metric_m(X, proj_m(lie_bracket(Z, Y)))) * Z for Z in m_on_basis])

def levi_civita(X, Y):
    """
    Estamos identificando X, Y con los campos de Killing asociados!!!
    """
    return -(1/2) * proj_m(lie_bracket(X, Y)) + U_tensor(X, Y)

In [20]:
# Chequeamos si la metrica es naturalmente reductiva
# long time
%time set([U_tensor(X, Y) == 0 for X in m_on_basis for Y in m_on_basis])

CPU times: user 6.39 ms, sys: 0 ns, total: 6.39 ms
Wall time: 6.43 ms


{True}

**Curvatura seccional:** para todos $X, Y \in \mathfrak m$
\begin{align*}
\langle R(X, Y) X, Y\rangle  = & -\frac34 \langle [X, Y]_{\mathfrak m}, [X, Y]_{\mathfrak m}\rangle - \frac12 \langle [X, [X, Y]_{\mathfrak m} ]_{\mathfrak m}, Y\rangle \\
& -\frac12 \langle [Y, [Y, X]_{\mathfrak m} ]_{\mathfrak m}, X\rangle + \langle U(X, Y), U(X, Y)\rangle \\
& - \langle U(X, X), U(Y, Y) \rangle + \langle Y, [ [X, Y]_{\mathfrak h}, X]_{\mathfrak m}\rangle
\end{align*}


In [21]:
def sec_curv(X, Y):
    sum1 = -(3/4) * metric_m(proj_m(lie_bracket(X, Y)), proj_m(lie_bracket(X, Y)))
    sum2 = -(1/2) * metric_m(proj_m(lie_bracket(X, proj_m(lie_bracket(X, Y)))), Y)
    sum3 = -(1/2) * metric_m(proj_m(lie_bracket(Y, proj_m(lie_bracket(Y, X)))), X)
    sum4 = metric_m(U_tensor(X, Y), U_tensor(X, Y))
    sum5 = -metric_m(U_tensor(X, X), U_tensor(Y, Y))
    sum6 = metric_m(Y, proj_m(lie_bracket(proj_h(lie_bracket(X, Y)), X)))
    return sum1 + sum2 + sum3 + sum4 + sum5 + sum6

In [22]:
# long time
for i in range(n):
    for j in range(i+1,n):
        print(sec_curv(m_on_basis[i], m_on_basis[j]))
print("")
print("Fin.")

1/2

Fin.


**Curvatura de Ricci:** Fijemos una base ortonormal $X_1, \ldots, X_n$ de $\mathfrak m$. Entonces para todo $X \in \mathfrak m$ se tiene
\begin{align*}
\operatorname{Ric}(X, X) = & \sum_i \langle R(X, X_i) X, X_i\rangle \\
= & -\frac12 \sum_i \langle [X, X_i]_{\mathfrak m}, [X, X_i]_{\mathfrak m} \rangle - \frac12 \sum_i \langle [X, [X, X_i]_{\mathfrak m} ]_{\mathfrak m}, X_i\rangle \\
& -\sum_i \langle [ X, [X, X_i]_{\mathfrak h} ]_{\mathfrak m}, X_i\rangle + \frac14 \sum_{i, j} \langle [X_i, X_j]_{\mathfrak m}, X\rangle^2 \\
&- \langle [Z, X]_{\mathfrak m}, X \rangle,
\end{align*}
en donde $Z = \sum_i U(X_i, X_i)$ (el cual puede calcularse implícitamente por $\langle Z, X\rangle = \operatorname{tr} \operatorname {ad}_X$)


In [23]:
# Implementacion menos eficiente
#def ricci_quadratic_form(X):
#    return sum([sec_curv(X, Y) for Y in m_on_basis])

In [24]:
sum_Uii = sum([U_tensor(X, X) for X in m_on_basis])
%time XiXj = {(i,j): proj_m(lie_bracket(m_on_basis[i], m_on_basis[j])).canonicalize_radical() for i in range(n) for j in range(n)}

CPU times: user 1.26 s, sys: 26.6 ms, total: 1.29 s
Wall time: 866 ms


In [25]:
def ricci_quadratic_form_1(X):
    XXi_m = [proj_m(lie_bracket(X, Y)) for Y in m_on_basis]
    XXXi_mm = [proj_m(lie_bracket(X, XXi_m[i])) for i in range(n)]
    XXXi_hm = [proj_m(lie_bracket(X,proj_h(lie_bracket(X, Y)))) for Y in m_on_basis]
    sum1 = -(1/2) * sum([metric_m(Y, Y) for Y in XXi_m])
    sum2 = -(1/2) * sum([metric_m(XXXi_mm[i], m_on_basis[i]) for i in range(n)])
    sum3 = -sum([metric_m(XXXi_hm[i], m_on_basis[i]) for i in range(n)])
    sum4 = (1/4) * sum([metric_m(XiXj[(i,j)], X)^2 for i in range(n) for j in range(n)])
    sum5 = -metric_m(proj_m(lie_bracket(sum_Uii, X)), X)
    return sum1 + sum2 + sum3 + sum4 + sum5

In [26]:
%time ricci_quadratic_form_1(m_on_basis[0])

CPU times: user 2.6 ms, sys: 0 ns, total: 2.6 ms
Wall time: 2.61 ms


1/2

In [27]:
#long time
for X in m_on_basis:
    print(ricci_quadratic_form_1(X))

1/2
1/2


Ahora definimos el tensor de Ricci, empezando por su forma cuadrática y luego polarizando

In [28]:
#def ricci_tensor(X, Y):
#    return (1/2) * (ricci_quadratic_form(X + Y) - ricci_quadratic_form(X) - ricci_quadratic_form(Y))

In [29]:
def ricci_tensor_1(X, Y):
    return (1/2) * (ricci_quadratic_form_1(X + Y) - ricci_quadratic_form_1(X) - ricci_quadratic_form_1(Y))

In [30]:
%time ricci_tensor_1(m_on_basis[0], m_on_basis[1])

CPU times: user 9.33 ms, sys: 0 ns, total: 9.33 ms
Wall time: 9.28 ms


0

In [31]:
Ric = matrix(SR,n,n)

# very long time
for i in range(n):
    for j in range(i,n):
        Ric[i,j] = Ric[j, i] = ricci_tensor_1(m_on_basis[i], m_on_basis[j]).canonicalize_radical()
        print('lugares ('+ str(i) + ',' +str(j) + ') y (' + str(j) + ',' + str(i) + ') completos...')
#        print(Ric)
        print(".............................")

lugares (0,0) y (0,0) completos...
.............................
lugares (0,1) y (1,0) completos...
.............................
lugares (1,1) y (1,1) completos...
.............................


In [32]:
Ric

[1/2   0]
[  0 1/2]

In [33]:
# long time, pero mejora mucho la eficiencia
%time Ric = matrix(n, n, lambda i, j: ricci_tensor_1(m_on_basis[i], m_on_basis[j])).canonicalize_radical()
Ric

CPU times: user 37 ms, sys: 28 μs, total: 37.1 ms
Wall time: 36.8 ms


[1/2   0]
[  0 1/2]

In [34]:
# Cáculo explícito de la curvatura seccional

gen_vars_m = [var("x"+str(i)) for i in range(len(m_on_basis))]
gen_vars_m_1 = [var("y"+str(i)) for i in range(len(m_on_basis))]
m_gen = sum([gen_vars_m[i] * m_on_basis[i] for i in range(len(m_on_basis))])
m_gen_1 = sum([gen_vars_m_1[i] * m_on_basis[i] for i in range(len(m_on_basis))])
m_gen, m_gen_1

(
[              0               0 -1/2*sqrt(2)*x0]
[              0               0 -1/2*sqrt(2)*x1]
[ 1/2*sqrt(2)*x0  1/2*sqrt(2)*x1               0],

[              0               0 -1/2*sqrt(2)*y0]
[              0               0 -1/2*sqrt(2)*y1]
[ 1/2*sqrt(2)*y0  1/2*sqrt(2)*y1               0]
)

In [35]:
sec_curv(m_gen, m_gen_1).factor()

1/2*(x1*y0 - x0*y1)^2

In [36]:
(sec_curv(m_gen, m_gen_1) / (metric_m(m_gen, m_gen) * metric_m(m_gen_1, m_gen_1) - metric_m(m_gen, m_gen_1)^2)).factor()

1/2

## La esfera $S^3 = \operatorname{SO}(4) / \operatorname{SO}(3)$

In [37]:
#Test round sphere
g_initial_basis = [
    matrix(SR,[[0,-1,0,0],[1,0,0,0],[0,0,0,0],[0,0,0,0]]),
    matrix(SR,[[0,0,-1,0],[0,0,0,0],[1,0,0,0],[0,0,0,0]]),
    matrix(SR,[[0,0,0,0],[0,0,-1,0],[0,1,0,0],[0,0,0,0]]),
    matrix(SR,[[0,0,0,-1],[0,0,0,0],[0,0,0,0],[1,0,0,0]]),
    matrix(SR,[[0,0,0,0],[0,0,0,-1],[0,0,0,0],[0,1,0,0]]),
    matrix(SR,[[0,0,0,0],[0,0,0,0],[0,0,0,-1],[0,0,1,0]]),
]
g_initial_basis

[
[ 0 -1  0  0]  [ 0  0 -1  0]  [ 0  0  0  0]  [ 0  0  0 -1]
[ 1  0  0  0]  [ 0  0  0  0]  [ 0  0 -1  0]  [ 0  0  0  0]
[ 0  0  0  0]  [ 1  0  0  0]  [ 0  1  0  0]  [ 0  0  0  0]
[ 0  0  0  0], [ 0  0  0  0], [ 0  0  0  0], [ 1  0  0  0],

[ 0  0  0  0]  [ 0  0  0  0]
[ 0  0  0 -1]  [ 0  0  0  0]
[ 0  0  0  0]  [ 0  0  0 -1]
[ 0  1  0  0], [ 0  0  1  0]
]

In [38]:
is_lie_algebra(g_initial_basis)

True

In [39]:
h_initial_basis = g_initial_basis[:3]
h_initial_basis

[
[ 0 -1  0  0]  [ 0  0 -1  0]  [ 0  0  0  0]
[ 1  0  0  0]  [ 0  0  0  0]  [ 0  0 -1  0]
[ 0  0  0  0]  [ 1  0  0  0]  [ 0  1  0  0]
[ 0  0  0  0], [ 0  0  0  0], [ 0  0  0  0]
]

In [40]:
def is_subalgebra(h_basis, g_basis):
    if is_lie_algebra(h_basis):
        for X in h_basis:
            if is_li(g_basis + [X]):
                return False
        return True

In [41]:
is_subalgebra(h_initial_basis, g_initial_basis)

True

In [42]:
def gram_schdmidt(basis, metric):
    on_basis = []
    for i, X in enumerate(basis):
        Y = X
        for j in range(i):
            Y -= metric(X, on_basis[j]) * on_basis[j]
        on_basis.append(1/sqrt(metric(Y, Y)) * Y)
    return on_basis

In [43]:
g_adapted_basis = h_initial_basis.copy()
for X in g_initial_basis:
    if is_li(g_adapted_basis + [X]):
        g_adapted_basis.append(X)

g_on_basis = gram_schdmidt(g_adapted_basis, biinv_metric)
h_on_basis = g_on_basis[:len(h_initial_basis)]

In [44]:
g_on_basis == g_initial_basis

False

In [45]:
g_on_basis

[
[           0 -1/2*sqrt(2)            0            0]
[ 1/2*sqrt(2)            0            0            0]
[           0            0            0            0]
[           0            0            0            0],

[           0            0 -1/2*sqrt(2)            0]
[           0            0            0            0]
[ 1/2*sqrt(2)            0            0            0]
[           0            0            0            0],

[           0            0            0            0]
[           0            0 -1/2*sqrt(2)            0]
[           0  1/2*sqrt(2)            0            0]
[           0            0            0            0],

[           0            0            0 -1/2*sqrt(2)]
[           0            0            0            0]
[           0            0            0            0]
[ 1/2*sqrt(2)            0            0            0],

[           0            0            0            0]
[           0            0            0 -1/2*sqrt(2)]
[           0            0            0            0]
[           0  1/2*sqrt(2)            0            0],

[           0            0            0            0]
[           0            0            0            0]
[           0            0            0 -1/2*sqrt(2)]
[           0            0  1/2*sqrt(2)            0]
]

In [46]:
m_initial_basis = g_on_basis[len(h_initial_basis):]
n = len(m_initial_basis)

In [47]:
def proj_h(X):
    return X - sum([biinv_metric(X, Y) * Y for Y in m_initial_basis])

def proj_m(X):
    return X - sum([biinv_metric(X, Y) * Y for Y in h_on_basis])

In [48]:
# Test con metrica normal homogenea
metric_m = biinv_metric

In [49]:
m_on_basis = gram_schdmidt(m_initial_basis, metric_m)
print(matrix(n, n, lambda i, j: metric_m(m_initial_basis[i], m_initial_basis[j])))
print("")
print(matrix(n, n, lambda i, j: metric_m(m_on_basis[i], m_on_basis[j])))

[1 0 0]
[0 1 0]
[0 0 1]

[1 0 0]
[0 1 0]
[0 0 1]


**Chequeamos la Ad-invariancia.** Por simplicidad trabajaremos al nivel del  álgebra de Lie verificando que $\operatorname{ad}_X |_{\mathfrak m}: \mathfrak m \to \mathfrak m$ sea antisimétrica para cada $X \in \mathfrak h$. Si denotamos por $X_1, \ldots, X_n$ la base ortonormal de $\mathfrak m$, entonces la matriz de $\operatorname{ad}_X$ en esta base esta dada por
$$(a_{ij}) = ( \langle [X, X_j], X_i\rangle)$$

In [50]:
def ad_matrix_m(X):
    return matrix(n, n, lambda i, j: metric_m(lie_bracket(X, m_on_basis[j]), m_on_basis[i]))

In [51]:
for X in h_on_basis:
    print(ad_matrix_m(X).is_skew_symmetric())

True
True
True


**Conexión de Levi-Civita:** para todos $X, Y \in \mathfrak m$, $$(\nabla_{X^*} Y^*)_o = - \frac12 [X, Y]_{\mathfrak m} + U(X, Y)$$ en donde $U: \mathfrak m \times \mathfrak m \to \mathfrak m$ está determinado implícitamente por la fórmula $$2 \langle U(X, Y), Z\rangle = \langle [Z, X]_{\mathfrak m}, Y \rangle + \langle X, [Z, Y]_{\mathfrak m}\rangle$$ para todo $Z \in \mathfrak m$


In [52]:
def U_tensor(X, Y):
    return (1/2) * sum([(metric_m(proj_m(lie_bracket(Z,X)),Y) + metric_m(X, proj_m(lie_bracket(Z, Y)))) * Z for Z in m_on_basis])

def levi_civita(X, Y):
    """
    Estamos identificando X, Y con los campos de Killing asociados!!!
    """
    return -(1/2) * proj_m(lie_bracket(X, Y)) + U_tensor(X, Y)

In [53]:
# Chequeamos si la metrica es naturalmente reductiva
# long time
%time set([U_tensor(X, Y) == 0 for X in m_on_basis for Y in m_on_basis])

CPU times: user 43.8 ms, sys: 3.14 ms, total: 46.9 ms
Wall time: 47 ms


{True}

**Curvatura seccional:** para todos $X, Y \in \mathfrak m$
\begin{align*}
\langle R(X, Y) X, Y\rangle  = & -\frac34 \langle [X, Y]_{\mathfrak m}, [X, Y]_{\mathfrak m}\rangle - \frac12 \langle [X, [X, Y]_{\mathfrak m} ]_{\mathfrak m}, Y\rangle \\
& -\frac12 \langle [Y, [Y, X]_{\mathfrak m} ]_{\mathfrak m}, X\rangle + \langle U(X, Y), U(X, Y)\rangle \\
& - \langle U(X, X), U(Y, Y) \rangle + \langle Y, [ [X, Y]_{\mathfrak h}, X]_{\mathfrak m}\rangle
\end{align*}


In [54]:
def sec_curv(X, Y):
    sum1 = -(3/4) * metric_m(proj_m(lie_bracket(X, Y)), proj_m(lie_bracket(X, Y)))
    sum2 = -(1/2) * metric_m(proj_m(lie_bracket(X, proj_m(lie_bracket(X, Y)))), Y)
    sum3 = -(1/2) * metric_m(proj_m(lie_bracket(Y, proj_m(lie_bracket(Y, X)))), X)
    sum4 = metric_m(U_tensor(X, Y), U_tensor(X, Y))
    sum5 = -metric_m(U_tensor(X, X), U_tensor(Y, Y))
    sum6 = metric_m(Y, proj_m(lie_bracket(proj_h(lie_bracket(X, Y)), X)))
    return sum1 + sum2 + sum3 + sum4 + sum5 + sum6

In [55]:
# long time
for i in range(n):
    for j in range(i+1,n):
        print(sec_curv(m_on_basis[i], m_on_basis[j]))
print("")
print("Fin.")

1/2
1/2
1/2

Fin.


**Curvatura de Ricci:** Fijemos una base ortonormal $X_1, \ldots, X_n$ de $\mathfrak m$. Entonces para todo $X \in \mathfrak m$ se tiene
\begin{align*}
\operatorname{Ric}(X, X) = & \sum_i \langle R(X, X_i) X, X_i\rangle \\
= & -\frac12 \sum_i \langle [X, X_i]_{\mathfrak m}, [X, X_i]_{\mathfrak m} \rangle - \frac12 \sum_i \langle [X, [X, X_i]_{\mathfrak m} ]_{\mathfrak m}, X_i\rangle \\
& -\sum_i \langle [ X, [X, X_i]_{\mathfrak h} ]_{\mathfrak m}, X_i\rangle + \frac14 \sum_{i, j} \langle [X_i, X_j]_{\mathfrak m}, X\rangle^2 \\
&- \langle [Z, X]_{\mathfrak m}, X \rangle,
\end{align*}
en donde $Z = \sum_i U(X_i, X_i)$ (el cual puede calcularse implícitamente por $\langle Z, X\rangle = \operatorname{tr} \operatorname {ad}_X$)


In [56]:
# Implementacion menos eficiente
#def ricci_quadratic_form(X):
#    return sum([sec_curv(X, Y) for Y in m_on_basis])

In [57]:
sum_Uii = sum([U_tensor(X, X) for X in m_on_basis])
%time XiXj = {(i,j): proj_m(lie_bracket(m_on_basis[i], m_on_basis[j])).canonicalize_radical() for i in range(n) for j in range(n)}

CPU times: user 125 ms, sys: 10.2 ms, total: 136 ms
Wall time: 114 ms


In [58]:
def ricci_quadratic_form_1(X):
    XXi_m = [proj_m(lie_bracket(X, Y)) for Y in m_on_basis]
    XXXi_mm = [proj_m(lie_bracket(X, XXi_m[i])) for i in range(n)]
    XXXi_hm = [proj_m(lie_bracket(X,proj_h(lie_bracket(X, Y)))) for Y in m_on_basis]
    sum1 = -(1/2) * sum([metric_m(Y, Y) for Y in XXi_m])
    sum2 = -(1/2) * sum([metric_m(XXXi_mm[i], m_on_basis[i]) for i in range(n)])
    sum3 = -sum([metric_m(XXXi_hm[i], m_on_basis[i]) for i in range(n)])
    sum4 = (1/4) * sum([metric_m(XiXj[(i,j)], X)^2 for i in range(n) for j in range(n)])
    sum5 = -metric_m(proj_m(lie_bracket(sum_Uii, X)), X)
    return sum1 + sum2 + sum3 + sum4 + sum5

In [59]:
%time ricci_quadratic_form_1(m_on_basis[0])

CPU times: user 12.3 ms, sys: 0 ns, total: 12.3 ms
Wall time: 11.7 ms


1

In [60]:
#long time
for X in m_on_basis:
    print(ricci_quadratic_form_1(X))

1
1
1


Ahora definimos el tensor de Ricci, empezando por su forma cuadrática y luego polarizando

In [61]:
#def ricci_tensor(X, Y):
#    return (1/2) * (ricci_quadratic_form(X + Y) - ricci_quadratic_form(X) - ricci_quadratic_form(Y))

In [62]:
def ricci_tensor_1(X, Y):
    return (1/2) * (ricci_quadratic_form_1(X + Y) - ricci_quadratic_form_1(X) - ricci_quadratic_form_1(Y))

In [63]:
%time ricci_tensor_1(m_on_basis[0], m_on_basis[1])

CPU times: user 37.5 ms, sys: 0 ns, total: 37.5 ms
Wall time: 37.3 ms


0

In [64]:
Ric = matrix(SR,n,n)

# very long time
for i in range(n):
    for j in range(i,n):
        Ric[i,j] = Ric[j, i] = ricci_tensor_1(m_on_basis[i], m_on_basis[j]).canonicalize_radical()
        print('lugares ('+ str(i) + ',' +str(j) + ') y (' + str(j) + ',' + str(i) + ') completos...')
#        print(Ric)
        print(".............................")

lugares (0,0) y (0,0) completos...
.............................
lugares (0,1) y (1,0) completos...
.............................
lugares (0,2) y (2,0) completos...
.............................
lugares (1,1) y (1,1) completos...
.............................
lugares (1,2) y (2,1) completos...
.............................
lugares (2,2) y (2,2) completos...
.............................


In [65]:
Ric

[1 0 0]
[0 1 0]
[0 0 1]

In [66]:
# long time, pero mejora mucho la eficiencia
%time Ric = matrix(n, n, lambda i, j: ricci_tensor_1(m_on_basis[i], m_on_basis[j])).canonicalize_radical()
Ric

CPU times: user 207 ms, sys: 180 μs, total: 207 ms
Wall time: 207 ms


[1 0 0]
[0 1 0]
[0 0 1]

In [67]:
# Cáculo explícito de la curvatura seccional

gen_vars_m = [var("x"+str(i)) for i in range(len(m_on_basis))]
gen_vars_m_1 = [var("y"+str(i)) for i in range(len(m_on_basis))]
m_gen = sum([gen_vars_m[i] * m_on_basis[i] for i in range(len(m_on_basis))])
m_gen_1 = sum([gen_vars_m_1[i] * m_on_basis[i] for i in range(len(m_on_basis))])
m_gen, m_gen_1

(
[              0               0               0 -1/2*sqrt(2)*x0]
[              0               0               0 -1/2*sqrt(2)*x1]
[              0               0               0 -1/2*sqrt(2)*x2]
[ 1/2*sqrt(2)*x0  1/2*sqrt(2)*x1  1/2*sqrt(2)*x2               0],

[              0               0               0 -1/2*sqrt(2)*y0]
[              0               0               0 -1/2*sqrt(2)*y1]
[              0               0               0 -1/2*sqrt(2)*y2]
[ 1/2*sqrt(2)*y0  1/2*sqrt(2)*y1  1/2*sqrt(2)*y2               0]
)

In [68]:
sec_curv(m_gen, m_gen_1).factor()

1/2*x1^2*y0^2 + 1/2*x2^2*y0^2 - x0*x1*y0*y1 + 1/2*x0^2*y1^2 + 1/2*x2^2*y1^2 - x0*x2*y0*y2 - x1*x2*y1*y2 + 1/2*x0^2*y2^2 + 1/2*x1^2*y2^2

In [69]:
(sec_curv(m_gen, m_gen_1) / (metric_m(m_gen, m_gen) * metric_m(m_gen_1, m_gen_1) - metric_m(m_gen, m_gen_1)^2)).factor()

1/2